In [1]:
import sys
import torch
import os
from os import makedirs
from argparse import ArgumentParser
from argparse import Namespace
from arguments import ModelParams, PipelineParams, OptimizationParams, get_combined_args
import numpy as np
from scene import Scene, GaussianModel
from utils.general_utils import safe_state
import open3d as o3d
from random import randint
from gaussian_renderer import render, network_gui
import matplotlib.pyplot as plt
from tqdm import tqdm

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
# Setup Command line Arg
sys.argv = ["train.py", "-s", "G:\\Universitat Siegen\\SA\\P-GPU\\Code\\gaussian-splatting\\data\\RGBD_Data", "-r" ,"1"]
                        # "-m" , "G:\\Universitat Siegen\\SA\\P-GPU\\Code\\gaussian-splatting\\output\\RGBD_Model"]

parser = ArgumentParser(description='Progressice Gaussian Splatting')
model = ModelParams(parser)
pipeline = PipelineParams(parser)
op = OptimizationParams(parser)

# parser.add_argument('--ip', type=str, default="127.0.0.1")
# parser.add_argument('--port', type=int, default=6009)
# parser.add_argument('--debug_from', type=int, default=-1)
# parser.add_argument('--detect_anomaly', action='store_true', default=False)
# parser.add_argument("--save_iterations", nargs="+", type=int, default=3000)
# parser.add_argument("--test_iterations", nargs="+", type=int, default=3000)
parser.add_argument("--quiet", action="store_true")
# parser.add_argument("--checkpoint_iterations", nargs="+", type=int, default=3000)
# parser.add_argument("--start_checkpoint", type=str, default = None)

args = parser.parse_args(sys.argv[1:])

# Initialize system state (RNG)
# safe_state(args.quiet)

dataset, iteration, pipeline = model.extract(args), op.extract(args), pipeline.extract(args)

In [3]:
with torch.no_grad():
    # Initialize Gaussians with Zero Tensor
    gaussians = GaussianModel(dataset.sh_degree)
    # Load Actual Gaussians, Camera from PCD
    scene = Scene(dataset, gaussians, shuffle=False)

Loading RGBD Scene
Loading Training Cameras
Loading Test Cameras
Number of points at initialisation :  2073600


In [4]:
# Pick a random Camera
viewpoint_stack = None
opt = op.extract(args)
if not viewpoint_stack:
    viewpoint_stack = scene.getTrainCameras().copy()


# viewpoint_cam = viewpoint_stack.pop(randint(0, len(viewpoint_stack)-1))
# viewpoint_cam = viewpoint_stack[0]
# # Render
# bg = torch.rand((3), device="cuda")

# render_pkg = render(viewpoint_cam, gaussians, pipeline, bg)

# image, viewspace_point_tensor, visibility_filter, radii = render_pkg["render"], render_pkg["viewspace_points"], render_pkg["visibility_filter"], render_pkg["radii"]


In [5]:
# image_numpy = image.cpu().detach().numpy()

# # Transpose the dimensions to [height, width, channels]
# image_numpy = np.transpose(image_numpy, (1, 2, 0))

# # Display the image using Matplotlib
# plt.imshow(image_numpy)
# plt.axis('off')  # Turn off axis numbers and ticks
# plt.show()

In [23]:
viewpoint_cam = viewpoint_stack[0]
print(viewpoint_cam.image_width, viewpoint_cam.image_height)
print(viewpoint_cam.world_view_transform.inverse())
print(viewpoint_cam.projection_matrix)
print(viewpoint_cam.full_proj_transform)
print(viewpoint_cam.camera_center)

# #save as PLY for verification
def saveTensorAsPLY(points,colors, file_name):
    point_cloud_np = points.cpu().detach().numpy() 
    # Create an Open3D point cloud
    point_cloud_o3d = o3d.geometry.PointCloud()
    point_cloud_o3d.points = o3d.utility.Vector3dVector(point_cloud_np)
    point_cloud_o3d.colors = o3d.utility.Vector3dVector(colors.cpu().detach().numpy())
    # Save the point cloud as a PLY file
    o3d.io.write_point_cloud(file_name, point_cloud_o3d)


1920 1080
tensor([[-1.0000e+00,  4.3711e-08, -7.5710e-08, -0.0000e+00],
        [-1.1384e-09,  8.6603e-01,  5.0000e-01, -0.0000e+00],
        [ 7.8368e-08,  5.0000e-01, -8.6603e-01,  0.0000e+00],
        [ 5.0000e-01,  1.0000e+00,  2.3000e+00,  1.0000e+00]], device='cuda:0')
tensor([[ 0.6174,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.5300,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  1.0001,  1.0000],
        [ 0.0000,  0.0000, -0.0100,  0.0000]], device='cuda:0')
tensor([[-6.1737e-01, -4.7917e-14,  8.7432e-08,  8.7423e-08],
        [ 2.6986e-08,  4.5902e-01,  5.0005e-01,  5.0000e-01],
        [-4.6741e-08,  2.6501e-01, -8.6611e-01, -8.6603e-01],
        [ 3.0868e-01, -1.0685e+00,  1.4820e+00,  1.4919e+00]], device='cuda:0')
tensor([0.5000, 1.0000, 2.3000], device='cuda:0')


In [24]:
ply_path = "G:\\Universitat Siegen\\SA\\P-GPU\\Code\\gaussian-splatting\\data\\RGBD_Data\\ply"
ply_files = os.listdir(ply_path)
viewpoint_cam = viewpoint_stack[0]

def loadPLY(ply_file):
       pcd = o3d.io.read_point_cloud(ply_file)
       points = np.asarray(pcd.points)
       colors = np.asarray(pcd.colors)
       points = torch.from_numpy(points).float().cuda()
       colors = torch.from_numpy(colors).float().cuda()
       return points, colors

def getPixelIndicesfromPLY(points, colors,  viewpoint_cam, sub_pixel_level=1):
       # Get Projection Matrix
       P = viewpoint_cam.full_proj_transform
       
       # Project points using projection matrix P in torch cuda and get result in NDC
       points_homogeneous = torch.cat((points, torch.ones(points.shape[0], 1, device=points.device)), dim=1)
       projected_points_homogeneous = torch.matmul(points_homogeneous, P.t()) # Clip space coordinates

       # # Filter points outside clip space
       # mask = (projected_points_homogeneous[:, 0] >= -1) & (projected_points_homogeneous[:, 0] < 1) & \
       #        (projected_points_homogeneous[:, 1] >= -1) & (projected_points_homogeneous[:, 1] < 1 ) # & \
              # (projected_points_homogeneous[:, 2] >= -1) & (projected_points_homogeneous[:, 2] < 1) // Removes points closer to camera so not included
       # projected_points_homogeneous = projected_points_homogeneous[mask]

       # Clip Space / Homogenous to NDC
       assert projected_points_homogeneous.shape[1] == 4
       # Extract x, y, z, w from the tensor
       x, y, z, w = projected_points_homogeneous[:, 0], projected_points_homogeneous[:, 1], projected_points_homogeneous[:, 2], projected_points_homogeneous[:, 3]
       x_ndc = x / w
       y_ndc = y / w
       z_ndc = z / w
       projected_points_NDC = torch.stack((x_ndc, y_ndc, z_ndc), dim=1)

       # print(projected_points_NDC.shape, projected_points_NDC.min(), projected_points_NDC.max())

       # NDC to Image Space [-1,1] to [0,1]
       projected_points_IS = (projected_points_NDC + 1) / 2

       # Filter points outside image space
       mask = (projected_points_IS[:, 0] >= 0) & (projected_points_IS[:, 0] < 1) & \
              (projected_points_IS[:, 1] >= 0) & (projected_points_IS[:, 1] < 1 ) # & \
              # (projected_points_IS[:, 2] >= 0) & (projected_points_IS[:, 2] < 1)
       projected_points_IS = projected_points_IS[mask]
       points_filtered = points[mask]
       colors_filtered = colors[mask]

       # print(projected_points_IS.shape, projected_points_IS.min(), projected_points_IS.max())

       # Get image width and height
       img_width, img_height = viewpoint_cam.image_width, viewpoint_cam.image_height
       # Calculate effective pixel width and height based on the sub-pixel variable
       effective_pixel_width, effective_pixel_height = img_width // sub_pixel_level, img_height // sub_pixel_level
       # Get pixel coordinates
       pixel_coordinates = projected_points_IS[:,:2] * torch.tensor([effective_pixel_width, effective_pixel_height], device=points.device).float()
       pixel_indices = torch.floor(pixel_coordinates).to(torch.int)

       # print(pixel_indices.shape, pixel_indices.min(), pixel_indices.max())
       return points_filtered,colors_filtered,pixel_indices

In [26]:
# Add new points from PLY
sub_pixel_level = 1

global_points = None
global_colors = None

for i in tqdm(range(len(viewpoint_stack))):
    viewpoint_cam = viewpoint_stack[i]
    # Check if PLY file exists
    ply_file_name = viewpoint_cam.colmap_id + ".ply"
    filtered_ply_file_name = viewpoint_cam.colmap_id + "_filtered.ply"

    # print(ply_file_name)

    if ply_file_name not in ply_files:
        continue

    ply_file_path = os.path.join(ply_path, ply_file_name)
    
    pcd_points, pcd_colors = loadPLY(ply_file_path)
    filtered_points, filtered_colors,  indices = getPixelIndicesfromPLY(pcd_points, pcd_colors, viewpoint_cam)
    
    # # Load Initial Points
    if global_points is None:
        print("Initialized with points from first Frame")
        global_points = filtered_points
        global_colors = filtered_colors
    else:
        global_filtered_points, global_filtered_colors, global_pixel_indices = getPixelIndicesfromPLY(global_points,global_colors, viewpoint_cam)

        # print(indices.shape, global_pixel_indices.shape)
        # Find indices of filtered_points that are not present in global_pixel_indices
        not_in_global = ~torch.isin(indices, global_pixel_indices).all(dim=1)
        
        # Get filtered_points that are not present in global_filtered_points
        filtered_points_not_in_global = filtered_points[not_in_global]
        filtered_colors_not_in_global = filtered_colors[not_in_global]

        print("Adding new points: ", filtered_points_not_in_global.shape)
        
        saveTensorAsPLY(filtered_points_not_in_global,filtered_colors_not_in_global, filtered_ply_file_name)
        # Append to global points
        global_points = torch.cat((global_points, filtered_points_not_in_global), dim=0)
        global_colors = torch.cat((global_colors, filtered_colors_not_in_global), dim=0)


saveTensorAsPLY(global_points,global_colors, "Global.ply")

  2%|▏         | 1/60 [00:00<00:30,  1.93it/s]

Initialized with points from first Frame


  5%|▌         | 3/60 [00:00<00:16,  3.53it/s]

Adding new points:  torch.Size([4434, 3])


  8%|▊         | 5/60 [00:01<00:14,  3.89it/s]

Adding new points:  torch.Size([4811, 3])


 12%|█▏        | 7/60 [00:01<00:12,  4.14it/s]

Adding new points:  torch.Size([8180, 3])


100%|██████████| 60/60 [00:02<00:00, 25.47it/s]

Adding new points:  torch.Size([25732, 3])


In [16]:
a = torch.Tensor([[1,2],[4,5]])
b = torch.Tensor([[3,4],[4,5],[0,9]])

mask = torch.isin(a,b)

mask_all_true = mask.all(dim=1)

mask_all_true

tensor([False,  True])